In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as 
import random
import os
import matplotlib.pyplot as plt
import cv2

import seaborn as sn
from sklearn.model_selection import train_test_split

import pathlib
from sklearn.metrics import classification_report

In [ ]:
maskdata = './Videos/Mask/'
nomaskdata = './Videos/NoMask/'
unsuredata = './Videos/Unsure/'

In [ ]:
nomask = pathlib.Path(nomaskdata).glob('**/*.jpg')
unsure = pathlib.Path(unsuredata).glob('**/*.jpg')

In [ ]:
X = []
y = []

# Get all of the no-mask pictures
for m in nomask:
    img = cv2.imread(str(m))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_img = cv2.resize(img,(224,224))
    X.append(resized_img)
    y.append(0)

for u in unsure:
    img = cv2.imread(str(u))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_img = cv2.resize(img,(224,224))
    X.append(resized_img)
    y.append(0)

# Get the number of pictures so we can undersample the mask pictures
numPictures = len(X)

# Get n random samples from the mask pictures where n is the number of no-mask pics
maskfilenames = random.sample(os.listdir(maskdata), numPictures)

for fname in maskfilenames:
    fullpath = maskdata+'/'+fname
    img = cv2.imread(fullpath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_img = cv2.resize(img,(224,224))
    X.append(resized_img)
    y.append(1)


X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8)

# Scale the pixel color data between 0 and 1
X_train = X_train/255
X_test = X_test/255

In [ ]:
# Check that the pictures are loaded correctly
num = 1

plt.imshow(X[num])
print(y[num])

In [ ]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(28,4,input_shape=(224,224,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(56,4,padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100,activation='relu'),
    
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

model.fit(X_train,y_train,epochs=10)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
pred = model.predict(X_test)
p = []
for i in pred:
    if i >= 0.5:
        p.append(1)
    else:
        p.append(0)

cm = tf.math.confusion_matrix(predictions=p, labels=y_test)

plt.figure(figsize=(10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Pred')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test,p))

In [ ]:
# Save the model

modelPath = './SavedModels/MaskDetection'

model.save(modelPath)

#model = tf.keras.models.load_model(modelPath)